In [1]:
# Import library-library
import os
import glob
import string

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
excluded_words = ["tempat", "waktu", "mencuri", "rampok"]

NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali", "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

In [3]:
def preprocessing(berita):
    # Preprocessing
    s = berita.lower()
    s = s.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    s = s.encode('ascii', 'replace').decode('ascii')
    ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", s).split())
    s.replace("http://", " ").replace("https://", " ")
    s = re.sub('\s+', ' ', s)
    s = s.strip()
    s = s.translate(str.maketrans("","", string.punctuation))
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    s = re.sub(r'\b\w{1,1}\b', '', s)
    tokens = [token for token in s.split(" ") if token != ""]

    # Stopwords checking
    T = [t for t in tokens if ((t in excluded_words) or (t not in NLTK_StopWords))]
    return T

In [5]:
df =pd.read_csv("scrapped_news/scrapped_news2.csv")
df.info()
# df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2028 non-null   object
 1   date         2028 non-null   object
 2   description  2028 non-null   object
 3   source       2028 non-null   object
dtypes: object(4)
memory usage: 63.5+ KB


In [6]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df = pd.read_csv("scrapped_news/scrapped_news2.csv")
df = df.dropna(subset=['description'], axis=0)
df.sort_values("description", inplace = True)
df.drop_duplicates(subset ="description", keep = False, inplace = True)

df.head()
df_train_unprocessed, df_test_unprocessed = train_test_split(df,test_size=0.1)

In [7]:
for x in range(0, df.shape[0]):
    text = preprocessing(df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [8]:
df.head()
df_train, df_test = train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
1872,\n Akhir Tragis Dini Nurdiani Dilaporka...,"Sabtu, 14 Mei 2022 22:15 WIB",warga cengkareng jakarta barat dinyatakan tewa...,www.detik.com
111,"\n Sempat Diduga Salah Tangkap, 4 Terda...","Senin, 25 Apr 2022 15:27 WIB",terdakwa terdakwa fikry abdul rohman randy apr...,www.detik.com
155,\n Ulah Haji Gadungan di Makassar Curi ...,"Jumat, 25 Mar 2022 07:00 WIB",basri haji gadungan ditangkap polisi wilayah b...,www.detik.com
1225,\n Kasus Narkotika dan Asusila Libatkan...,"Rabu, 06 Apr 2022 09:19 WIB",ketua pn bengkulu jon sarman saragih perkara p...,www.detik.com
1549,\n Rampok-Paksa Wanita Buka Baju Modus ...,"Kamis, 02 Des 2021 11:56 WIB",wanita labuhanbatu utara labura sumatera utara...,www.detik.com


In [9]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, "corpus/model/desc_text_test_unprocessed.pkl")

['corpus/model/desc_text_test_unprocessed.pkl']

In [10]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [11]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  203
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  1825


In [12]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total.csv', index=False)
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [13]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(203, 5444)
tfidf_train_matrix
(1825, 18476)


In [14]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train.pkl")

['corpus/matrix/tfidf_train.pkl']

### Pencarian 

In [15]:
df_total = pd.read_csv('corpus/dataset/df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2028 entries, 0 to 2027
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2028 non-null   object
 1   date         2028 non-null   object
 2   description  2028 non-null   object
 3   source       2028 non-null   object
dtypes: object(4)
memory usage: 79.2+ KB
None
------------------------------------------------------------------------------------------
203
1825


In [16]:
kueri='curi'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['curi']


Top 10 most similar documents in corpus:
No ID Dokumen  :  1223
Tanggal        :  Jumat, 20 Mei  2022 17:20 WIB
Isi berita     :  polisi menetapkan jahoor ul hassan tersangka pembunuhan juju juariyah 46 alias mamih juju warga kabupaten tasikmalaya dendam motif pria bunuh mamih juju tersangka warga negara pakistan mantan suami korban perbuatan kejinya pria 42 mendekam tahanan polres tasikmalaya kota motifnya dendam hasil pendalaman pemeriksaan saksisaksi ditemukan tersangka dendam dicerai dendam dibagi hasil usaha harta gonogini korban tersangka bercerai kabid humas polda jabar ibrahim tompo didampingi kapolres tasikmalaya kota akbp aszhari kurniawan mapolres jumat 2052022 kejadian pembunuhan ibrahim diawali kedatangan jahoor ruko milik mamih juju kedatangan jahoor mengajak rujuk pembagian hasil usaha gono gini bercerai ibrahim perbincangan diduga berujung kekesalan jahoor keinginannya dipenuhi korban pelaku aksi kekerasan korban meninggal dunia ibrahim terkait status kewarga